<img src="./images/DLI_Header.png" style="width: 400px;">

# 遷移學習

目前為止，我們在大型資料集上訓練出準確的模型，也下載了預先訓練的模型，讓我們不需要訓練就能使用。但如果我們找不到符合需求的預先訓練模型，又沒有夠大的資料集來從零開始訓練模型時，該怎麼辦？在這種情況下，我們可以使用一種非常有用的技術，也就是[遷移學習](https://blogs.nvidia.com/blog/2019/02/07/what-is-transfer-learning/)。

透過遷移學習，我們可以重新訓練預先訓練模型，來進行與原本的訓練工作有重疊的工作。用比較容易理解的比喻來說，就像是已經很擅長使用某種媒介如水彩的藝術家，想要學習應用另一種媒材，例如用炭筆作畫。我們可以合理推測，藝術家繪製水彩畫時所學到的技能，會對學習如何使用炭筆作畫大有幫助。

以深度學習的例子來說，假設我們有個預先訓練的模型，非常擅於辨識不同類型的汽車，而現在我們想要訓練出可以辨識摩托車類型的模型。這時辨識汽車模型的很多學習結果就會非常實用，例如辨識頭燈和車輪的能力。

當我們沒有大規模且多樣的資料集時，遷移學習特別能發揮強大的效用。在這種情況下，從零開始訓練的模型也許能快速地記憶訓練資料，但卻無法對新資料進行有效歸納。透過遷移學習，你更有機會運用小型資料集訓練出準確且穩定的模型。

## 目標

* 為遷移學習準備預先訓練模型
* 在預先訓練模型上使用自己的小型資料集執行遷移學習
* 進一步微調模型以取得更佳效能

## 個人化狗門

在我們的上一個練習中，我們使用了預先訓練的 [ImageNet](http://www.image-net.org/) 模型，讓所有狗都能進入，但把其他動物排除在外。在本練習中，我們想要建立僅供特定狗種使用的狗門。因此，我們要為名為 Bo 的狗製作自動狗門，Bo 就是 2009 到 2017 年時任美國總統所養的「第一狗狗」。`presidential_doggy_door`資料夾中有更多 Bo 的相片。

<img src="presidential_doggy_door/train/bo/bo_10.jpg">

這裡的難題是預先訓練的模型沒有經過辨識這隻狗的訓練，而且我們只有 30 張 Bo 的相片。如果我們試著用這 30 張相片從零開始訓練模型，結果就會是過度擬合和缺乏歸納能力。不過，如果我們把擅於偵測狗類的預先訓練模型當作起點，就可以運用這個學習結果，來透過我們的小型資料集讓模型對 Bo 有歸納性的理解。我們可以用遷移學習解決這項難題。

## 下載預先訓練模型

[ImageNet 預先訓練模型](https://keras.io/api/applications/vgg/#vgg16-function)通常是電腦視覺遷移學習的絕佳選擇，因為這些模型已學會分類各種不同類型的影像。在學習過程中，這些模型已經學會偵測許多不同類型的[特徵](https://developers.google.com/machine-learning/glossary#)，這對影像識別相當有用。由於 ImageNet 模型已經學會偵測動物，包含狗，因此特別適合用於這項偵測 Bo 的遷移學習工作。

首先讓我們下載預先訓練的模型。同樣地，這也可以直接從 Keras 函式庫取得。在下載過程中，會有一項很關鍵的差異。ImageNet 模型的最後一層是 1000 單位的[密集層](https://developers.google.com/machine-learning/glossary#dense-layer)，表示資料集有 1000 種可能的類別。在我們的案例中，我們想要建立不同的分類方式：這"是"或"不是" Bo？由於我們想使用不同的分類方式，所以必須移除模型的最後一層。如果要這麼做，我們可以在下載模型時設定旗標 `include_top=False`。移除最後一層之後，我們可以新增一層來產生我們想要的分類類型：

In [ ]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False)

In [ ]:
base_model.summary()

## 凍結基本模型
將新的模型層新增到[預先訓練的模型](https://developers.google.com/machine-learning/glossary#pre-trained-model)之前，我們必須採取一個很重要的步驟：凍結模型的預先訓練層。這表示在訓練時，我們不會更新預先訓練模型的基本層。我們只會更新為了新分類而新增的最後一層。之所以要凍結初始模型層，是因為我們想要保留在 ImageNet 資料集上進行訓練所取得的學習結果。如果在這個階段沒有凍結這些模型層，我們很有可能會破壞有價值的資訊。稍後在微調的流程中，會有選項可以取消凍結和訓練這些模型層。

凍結基本層非常簡單，只需要在模型上把可訓練設定為 `False`。

In [ ]:
base_model.trainable = False

## 新增模型層

現在我們可以將新的可訓練層加入預先訓練的模型。這些可訓練層會從預先訓練層中取得特徵，並將特徵轉換為對新資料集的預測。我們會在模型上新增兩層。第一層就是我們先前在[卷積神經網路](https://developers.google.com/machine-learning/glossary#convolutional_layer)中看到的池化層。(如果你想要更徹底瞭解池化層在卷積神經網路中扮演的角色，請閱讀[這篇詳細的部落格文章](https://machinelearningmastery.com/pooling-layers-for-convolutional-neural-networks/#:~:text=A%20pooling%20layer%20is%20a,Convolutional%20Layer))。接著我們要新增最後一層，用於分類是或不是 Bo。這會是具有單一輸出的密集連結層。

In [ ]:
inputs = keras.Input(shape=(224, 224, 3))
# Separately from setting trainable on the model, we set training to False 
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

讓我們來看看模型，現在我們已經將預先訓練的模型與新的層結合在一起。

In [ ]:
model.summary()

Keras 在此提供的摘要很有幫助，因其將 vgg16 預先訓練模型顯示為單一單位，而不是顯示所有的內部層。另外也值得注意的是，我們有許多不可訓練的參數，因為我們已經凍結預先訓練的模型。

## 編寫模型

與先前的練習一樣，我們要使用損失與指標選項來編寫模型。我們必須在這裡選擇一些不同的選項。在先前的案例中，我們的分類問題會用到多種類別。因此，我們會選擇分類交叉熵來計算我們的損失。在這個案例中，我們只是要處理二元分類問題 (是 Bo 或不是 Bo)，因此要使用[二元交叉熵](https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy)。若要進一步瞭解兩者之間的差異，請參閱[此處](https://gombru.github.io/2018/05/23/cross_entropy_loss/)。我們也會採用二元準確度，而非傳統的準確度。

經由設定 `from_logits=True`，我們等於是在通知[損失函數](https://gombru.github.io/2018/05/23/cross_entropy_loss/)輸出值不需要被正規化 (例如透過 softmax)。

In [ ]:
# Important to use binary crossentropy and binary accuracy as we now have a binary classification problem
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=[keras.metrics.BinaryAccuracy()])

## 增強資料

由於現在我們要處理的資料集非常小，所以增強資料尤為重要。和先前一樣，我們要對現有影像做細微修改，讓模型可以從更多不同的影像學習。這有助於模型學會辨識 Bo 的新圖片，而不只是記憶訓練時使用的圖片。

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# create a data generator
datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) # we don't expect Bo to be upside-down so we will not flip vertically

## 載入資料

目前我們已經看過幾種不同的格式的資料集。在 MNIST 練習中，我們可以直接從 Keras 函式庫下載資料集。以手語資料集而言，資料位於 CSV 檔案中。在本練習中，我們要使用 Keras 的 [`flow_from_directory`](https://keras.io/api/preprocessing/image/)函數，直接從資料夾載入影像。我們建立了目錄來協助這個流程順利進行。在 `presidential_doggy_door`目錄中，我們有訓練和驗證目錄，兩個目錄都有分別包含 Bo 的影像和非 Bo 的影像的資料夾。在 not_bo 目錄中，有其他狗和貓的圖片，用來讓模型學會排除其他寵物。歡迎你進一步探索影像，以瞭解我們的資料集。

請注意，[flow_from_directory](https://keras.io/api/preprocessing/image/) 也讓我們可以調整影像大小來符合模型：244x244 像素和 3 個色頻。

In [ ]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('presidential_doggy_door/train/', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='binary', 
                                       batch_size=8)
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('presidential_doggy_door/valid/', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode='binary', 
                                      batch_size=8)

## 訓練模型

讓我們來訓練模型並看看結果如何。請回想一下使用資料產生器的方式，我們必須要明確設定 `steps_per_epoch`的數量：

In [ ]:
model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=20)

## 結果討論

訓練與驗證準確度應該都相當高。結果很出色！雖然我們只能在小型資料集上進行訓練，不過因為有從 ImageNet 模型遷移的知識，模型可以達到高準確度和具備良好的歸納能力。這表示模型能清楚辨識 Bo 和不是 Bo 的寵物。

如果你發現驗證準確度有些波動，那也沒關係。我們會在下一節提供改善模型的技術。

## 微調模型

現在模型的新層經過訓練後，我們可以選擇應用最後一項技巧來改善模型，也就是[微調](https://developers.google.com/machine-learning/glossary#f)。微調時我們要先取消凍結整個模型，並採用非常小的[學習率](https://developers.google.com/machine-learning/glossary#learning-rate)再次訓練模型。這樣一來，預先訓練的基本層會有非常微量的進步並經過稍微調整，以小幅度的方式改善模型。

請注意，有凍結層的模型一定要經過完整訓練後，才執行此步驟。我們先前新增至模型的未經訓練池化層和分類層，都是經過隨機初始化。這表示這兩層需要大幅更新才能正確分類影像。在整個[反向傳播](https://developers.google.com/machine-learning/glossary#backpropagation)過程中，最後一層的大型初始更新會導致預先訓練層也需要大幅更新。這些更新可能會破壞預先訓練的重要功能。然而，在最後兩層經過訓練且聚合的情況下，對整體模型的任何更新都會小得多 (尤其是非常小的學習率)，而不會破壞前幾層的功能。

讓我們試著取消凍結預先訓練的模型層，然後微調模型：

In [ ]:
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are taken into account
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),  # Very low learning rate
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=10)

## 檢查預測結果

現在我們的模型已經過完整訓練，可以為 Bo 建立狗門了！首先我們來看看模型的預測。我們要用和上次建立狗門相同的方式來預先處理影像。

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.imagenet_utils import preprocess_input

def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image)

def make_predictions(image_path):
    show_image(image_path)
    image = image_utils.load_img(image_path, target_size=(224, 224))
    image = image_utils.img_to_array(image)
    image = image.reshape(1,224,224,3)
    image = preprocess_input(image)
    preds = model.predict(image)
    return preds

用幾張影像試試看，然後看看預測結果如何：

In [ ]:
make_predictions('presidential_doggy_door/valid/bo/bo_20.jpg')

In [ ]:
make_predictions('presidential_doggy_door/valid/not_bo/121.jpg')

看來負數預測結果表示是 Bo，而正數預測結果則表示是其他寵物。我們可以用這項資訊來控制狗門只讓 Bo 進來！

## 練習：Bo 的狗門

請填入下列程式碼以實作 Bo 的狗門：

In [ ]:
def presidential_doggy_door(image_path):
    preds = make_predictions(image_path)
    if FIXME:
        print("It's Bo! Let him in!")
    else:
        print("That's not Bo! Stay out!")

## 解決方案

按一下以下的「...」來查看解決方案。

```python
def presidential_doggy_door(image_path):
    preds = make_predictions(image_path)
    if preds[0] < 0:
        print("It's Bo! Let him in!")
    else:
        print("That's not Bo! Stay out!")
```

來試試看吧！

In [ ]:
presidential_doggy_door('presidential_doggy_door/valid/not_bo/131.jpg')

In [ ]:
presidential_doggy_door('presidential_doggy_door/valid/bo/bo_29.jpg')

## 摘要

做得好！透過遷移學習，你使用非常小的資料集就打造出高度準確的模型。這可以是極為強大的技術，也能決定一項專案是成功還是無法實現。我們希望這些技術能在未來的類似案例中幫助到你！

在 [NVIDIA 遷移學習工具組](https://developer.nvidia.com/tlt-getting-started)中有豐富的遷移學習相關實用資源。

### 清除記憶體
在繼續之前，請執行下列儲存格以清除 GPU 記憶體。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## 下一步

到目前為止，此實作坊的重點主要是影像分類。在下一節中，為了更全面地介紹深度學習，我們要切換主題並討論與序列資料相關的作業，這需要用到不同的方法。

請繼續前往下一節：[*序列資料*](./06_headline_generator.ipynb)。